##  Install Required Packages

In [ ]:
pip install pandas openpyxl mysql-connector-python

## Import Libraries

In [ ]:
import pandas as pd
import mysql.connector
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

## Connect to MySQL and Run Query

In [ ]:
# Database connection

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Enter your password here",
    database="sakila"
)

query = """
SELECT f.title, COUNT(r.rental_id) AS rental_count
FROM rental r
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
GROUP BY f.film_id, f.title
ORDER BY rental_count DESC
LIMIT 5;
"""

df = pd.read_sql(query, db)


## Save Data to Excel

In [ ]:
# Save report to Excel
output_file = r"E:\reports\customers_report.xlsx"
df.to_excel(output_file, index=False, startrow=3)  # start at row 4

#  Open workbook to format

wb = load_workbook(output_file)
ws = wb.active

# Add Report Title
ws.merge_cells('A1:B1')  # Merge first two columns for title
title_cell = ws['A1']
title_cell.value = "Top 5 Rented Movies Report"
title_cell.font = Font(size=14, bold=True)
title_cell.alignment = Alignment(horizontal='center')

# Format header row (row 4 because DataFrame starts at row 4)
for cell in ws[4]:
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal='center')

# Optional: add autofilter
ws.auto_filter.ref = ws.dimensions

# Save formatted workbook
wb.save(output_file)
print(f"Formatted Excel report saved at {output_file}")

## Email the Excel Report

In [ ]:
# Email settings
sender_email = "abc@gmail.com"       # Replace with your email
receiver_email = "xyz@gmail.com"     # Replace with recipient
email_password = "123645"    # Replace with your email password

subject = "Daily Customers Report"
body = "Hi Team,\n\nPlease find attached the daily customers report.\n\nRegards,\nAkanksha"

# Create email message
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = receiver_email
msg['Subject'] = subject
msg.attach(MIMEBase('application', 'octet-stream'))

# Attach the Excel file
part = MIMEBase('application', "octet-stream")
with open(output_file, "rb") as file:
    part.set_payload(file.read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', f'attachment; filename= {output_file.split("\\")[-1]}')
msg.attach(part)

# Send email
with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
    server.login(sender_email, email_password)
    server.sendmail(sender_email, receiver_email, msg.as_string())

print("Email sent successfully!")
